## GPT와 streamlit을 연동해보자!

In [1]:
# API Key, DB 자격 증명 및 환경 변수와 같은 구성 설정을 안전하게 관리하는 python 패키지 설치
!pip install python-decouple

In [2]:
!pip install langchain-openai pydantic

In [8]:
%%writefile module/myChatApp.py

import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

chat_model = ChatOpenAI(model_name='gpt-3.5-turbo',                # fine-tuning된 모델명 넣어도 됨
                        api_key = st.secrets["OPENAI_API_KEY"],    # secrets.toml파일에 저장되어 있는 key값으로 value에 접근
                        temperature=0.5
                       )

my_prompt = PromptTemplate(input_variables=["chat_history", "question"],
                        template="""You are a AI assistant.
                        You are currently having a conversation with a human.
                        Answer the questions.
                        chat_history: {chat_history},
                        Human: {question}
                        AI assistant:
                        """
                       )

# 일반적인 코드에서는 memory 객체를 생성하면 대화 내용들을 기억하지만 streamlit에서는 웹 서버에서 요청, 응답을 수행하기 때문에
# 세션에 저장하지 않으면 다 초기화가 됨(따라서 memory 객체를 세션에서 관리해주는 코드를 작성해야 함)
if "pre_memory" not in st.session_state :
    # 세션에 pre_memory라는 key의 value값으로 ConversationBufferMemory객체를 저장해서 활용
    st.session_state['pre_memory'] = ConversationBufferMemory(memory_key="chat_history",
                                                              return_messages=True
                                                             )

llm_chain = LLMChain(llm=chat_model,
                     prompt= my_prompt,
                     memory=st.session_state['pre_memory']   # 세션으로 관리하고 있는 메모리 클래스 입력
                    )

# ChatGPT 서비스와 유사하게 웹 상에서 우리의 질의 응답 내역이 계속 보여져야 하기 때문에 세션관리가 필요
if "messages" not in st.session_state :
    st.session_state["messages"] = [
        # 맨 처음 웹 화면을 켰을때 출력될 수 있는 기본 문구를 설정
        {"role":"assistant", "content":"안녕하세요, 저는 AI Assistant 입니다 :"}
    ]

# ============================================웹 화면 표시부분===============================================

st.title("나의 작고 소중한 GPT 챗봇ㅋ")

# 반복문으로 messages에 있는 모든 대화 기록에 접근
for message in st.session_state["messages"] :
    # chat_message : 메시지의 발신자 role(assistant인지 user인지)에 따라 UI를 구분하여 메시지 창을 표시해주는 함수
    with st.chat_message(message["role"]) :   # 세션에 저장된 각 message의 role에 접근(맨 처음에는 assistant만 있는 상태)
        st.write(message["content"])          # 해당 role에 맞는 content값 출력

# chat_input : 채팅 메시지를 입력하는 UI를 생성하고 사용자가 입력한 텍스트를 변수에 저장
user_prompt = st.chat_input("사용자 입력 ><")

# 사용자가 텍스트를 입력했을 때
if user_prompt is not None :
    st.session_state['messages'].append({"role":"user", "content":user_prompt})
    with st.chat_message("user") :      # user로 메시지 창 UI 출력
        st.write(user_prompt)           # 사용자가 입력한 택스트 출력
        
# assistant가 응답하는 코드
# 마지막 메시지의 role이 assistant가 아니라면 새로운 답변을 생성하고 세션에 추가
# (마지막 메시지를 보는 것은 assistant가 본인의 응답에도 계속 대단하는 자문자답을 방지하기 위한 용도)
if st.session_state["messages"][-1]["role"] != "assistant" :
    with st.chat_message("assistant") :
        # llM이 응답을 생성하는 동안 로딩 애니메이션 실행
        with st.spinner("응답 생성중...") :
            try :
                ai_response = llm_chain.predict(question=user_prompt)  # llm 응답 생성  
                st.write(ai_response)                                  # 응답 출력
                st.session_state["messages"].append({"role":"assistant", "content":ai_response})  # 세션에 응답 내용 저장

            # 발생한 에러 출력
            except Exception as e :
                st.error(f"LLM 에러 발생 : {e}")


#===========================================================================================================

Overwriting module/myChatApp.py
